In [2]:
!pip install fitz
!pip install PyMuPDF
!pip install PyPDF2
!pip install transformers

import nltk
nltk.download("punkt")

     |████████████████████████████████| 3.2MB 23.4MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 133kB 56.5MB/s 
     |████████████████████████████████| 430kB 55.9MB/s 
     |████████████████████████████████| 235kB 57.8MB/s 
     |████████████████████████████████| 573kB 53.3MB/s 
     |████████████████████████████████| 5.5MB 52.2MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34547 sha256=fdbfb4a3812417a94426171831ca10d6f9d958e576b718344dc24880605a139d
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
  Created wheel for pyxnat: filename=pyxnat-1.4-cp36-none-any.whl size=92689 sha256=25bbde94dda48969127cde4b5e8e5448550622e6a62549b5333439d5e415fd48
  Stored in directory: /root/.cache/pip/wheels/db/64/83/16e08dbf817bc92dfb70dd9b0d5df4e3d7ff2867af68fd08af
  Created wheel for traits: filenam

True

In [3]:
import re
import PyPDF2
import sys
import fitz
import json
from nltk.tokenize import sent_tokenize, word_tokenize

def parseIndexText(text):
    '''
    Given Index of the format
    `Cloud Computing ................. 74, 81`
    It will extract as ["Concept Page Numbers", ..]
    '''
    contents = text.splitlines()
    content_with_pns = []
    for i in range(len(contents)):
        # check if the next line is numbers only
        if (len(content_with_pns) > 1) and (re.search('[0-9]+',content_with_pns[-1]) is None):
            content_with_pns[-1] =  content_with_pns[-1] + " " + contents[i]
        elif re.search('[a-zA-Z]+', contents[i]) is not None:   
            # if not add to list
            content_with_pns.append(contents[i])
        elif len(content_with_pns) > 1:
            # if yes append to last element for format `Cloud Computing ...... 74, 81 \n 32,22`
            temp = content_with_pns[-1]  
            temp = temp + contents[i]
            content_with_pns[-1] = temp
    return content_with_pns

    
def extractConceptsFromIndexPage(content_with_pns):
    '''
    Extract concepts given the index page of a text book
    '''
    concept_dict = {}

    for content in content_with_pns:
        concepts = content.split(',')
        curr_concepts = []
        curr_pages = []
        for concept in concepts:
            # name of the concepts
            if re.search('[a-zA-Z]+', concept) is not None:
                curr_concepts.append(concept.strip())
            else:
            # page number of concept
                concept = concept.strip()
                if len(concept) > 0:
                    curr_pages.append(concept.replace("–","-").strip())

        if len(curr_pages) > 0:
            concept_dict.update({curr_concept:curr_pages for curr_concept in curr_concepts})

    return concept_dict


# Entry Point
def extractConceptFromIndex(path='', filename="Cloud Computing Bible.pdf", indexPages=(497,527), output=""):
    '''
    To load data from pdf
    Args : filename, name of pdf file
    Rets : the list of concepts from the PDF
    '''
    # Ref : https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
    doc = fitz.open(filename)
    all_concepts_dict = {}

    for pageNumber in range(*indexPages):
        page = doc.loadPage(pageNumber)
        text = page.getText()
        content_with_pns = parseIndexText(text)
        all_concepts_dict.update(extractConceptsFromIndexPage(content_with_pns))
    
    json_dict = json.dumps(all_concepts_dict)
    f = open(output + filename+"_concepts.json", "w+")
    f.write(json_dict)
    f.close()


    # return {concepts:[page numbers, ....]}
    concept_names_raw = list(all_concepts_dict.keys())
    concept_names = []
    for concept_name in concept_names_raw:
        if (concept_name not in concept_names) and (concept_name+"s" 
                    not in concept_names) and (concept_name[:-1] not in concept_names):
            concept_names.append(concept_name)

    return concept_names

# concepts = extractConceptFromIndex(path="AL-CPL\\textbooks\\", filename='Networking.pdf')
# print(concepts)
# print(len(concepts))
concept_names = extractConceptFromIndex()

In [4]:
content_page_nos = [26,496]
total_text = []
doc = fitz.open("Cloud Computing Bible.pdf")
for pageNumber in range(*content_page_nos):
        page = doc.loadPage(pageNumber)
        text = page.getText()
        total_text.append(text)

processed_text = []
for text in total_text:
    text = text.replace("\n", "").lower()
    for line in sent_tokenize(text):
        processed_text.append(line)

dataset = []
for line in processed_text:
    for concept in concept_names:
        if concept in line:
            dataset.append([concept, line])

In [3]:
for concept in concept_names:
  if concept in dataset[0]:
    print(concept)

dataset[0]

cloud computing


['cloud computing',
 'xxvin the five months that i have been researching and writing cloud computing bible, it has become clear to me that most people recognize that cloud computing is a big deal, even if they are not really clear why that is so.']

In [5]:
# O: 0, B: 1, I: 2

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

MAX_LENGTH = 80

def tokenizerfnc(str1):
    return tokenizer.encode(str1, max_length=MAX_LENGTH, pad_to_max_length=True, truncation=True, add_special_tokens=True)

final_dataset = []
for idx, sent in enumerate(dataset):
    concept = sent[0]
    labels_sent = [0]*MAX_LENGTH
    concept_b = concept.split(" ")[0]
    concept_i = concept.split(" ")[1:]
    tokenized_words = tokenizer.tokenize(sent[1])
    for idx1 in range(min(len(tokenized_words), MAX_LENGTH)):
        if tokenized_words[idx1] == concept_b and (idx1+1 < MAX_LENGTH):
            labels_sent[idx1 + 1] = 1
        elif tokenized_words[idx1] in concept_i and (idx1+1 < MAX_LENGTH):
            labels_sent[idx1 + 1] = 2
    dataset[idx].append(tokenizer.tokenize(sent[1]))
    final_dataset.append([tokenizerfnc(sent[1]), labels_sent])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [9]:
len1 =0 
total = 0
for i in dataset:
  len1 += len(i[2])
  total += 1
print(len1/total)

70.41286930395593


In [7]:
print(dataset[0])
print("\n")

print(tokenizer.tokenize(dataset[0][1]))
print(len(tokenizer.tokenize(dataset[0][1])))
print("\n")

print(final_dataset[0])
print("\n")

print(len(final_dataset))
print("\n")

['cloud computing', 'xxvin the five months that i have been researching and writing cloud computing bible, it has become clear to me that most people recognize that cloud computing is a big deal, even if they are not really clear why that is so.', ['xx', '##vin', 'the', 'five', 'months', 'that', 'i', 'have', 'been', 'researching', 'and', 'writing', 'cloud', 'computing', 'bible', ',', 'it', 'has', 'become', 'clear', 'to', 'me', 'that', 'most', 'people', 'recognize', 'that', 'cloud', 'computing', 'is', 'a', 'big', 'deal', ',', 'even', 'if', 'they', 'are', 'not', 'really', 'clear', 'why', 'that', 'is', 'so', '.']]


['xx', '##vin', 'the', 'five', 'months', 'that', 'i', 'have', 'been', 'researching', 'and', 'writing', 'cloud', 'computing', 'bible', ',', 'it', 'has', 'become', 'clear', 'to', 'me', 'that', 'most', 'people', 'recognize', 'that', 'cloud', 'computing', 'is', 'a', 'big', 'deal', ',', 'even', 'if', 'they', 'are', 'not', 'really', 'clear', 'why', 'that', 'is', 'so', '.']
46


[[10

In [35]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from transformers import BertModel

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ModelDataset(Dataset):

    def __init__(self, data):
        self.data = [torch.LongTensor(i[0]) for i in data]
        self.labels = [torch.LongTensor(i[1]) for i in data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self,i):
        a = self.data[i]
        b = self.labels[i]
        return a, b

class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        
        self.output_size = 3
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')

        for param in self.bert_layer.parameters():
          param.requires_grad = False
        
        self.rnn = nn.LSTM(input_size = 768, hidden_size=256, num_layers=1, bidirectional=True)

        self.scoring = nn.Linear(2*256,self.output_size)

    def forward(self, x, attn_masks): #N x L
        
        l = self.bert_layer(x, attention_mask = attn_masks) #N x L x E
        x = l[0].permute(1,0,2) #L x N x E

        hidden = None
        output_lstm,hidden = self.rnn(x,hidden) #L x N x H
        output_lstm_flatten = output_lstm.view(-1,256*2) #(L*N) x H
        output_flatten = self.scoring(output_lstm_flatten) #(L*N) x O
        return output_flatten.view(x.shape[1],x.shape[0],self.output_size) #N x L x O

In [36]:
def train_epoch(model, loader, criterion, optimizer):
  model.train()
  flag = 0
  for batch_idx, (data, labels) in enumerate(loader):
      
      data = data.to(DEVICE)
      labels = labels.to(DEVICE)

      optimizer.zero_grad()
      attn_masks = (data != 0).long()
      attn_masks = attn_masks.to(DEVICE)
      out = model(data, attn_masks)
      _, predicted = torch.max(out.data, 2)
      if flag == 0:
        torch.set_printoptions(profile="full")
        #print(predicted.shape)
        #print(predicted)
        #print("hellotrain: ", torch.nonzero(predicted).shape[0] == 0)
        flag = 1

      loss = criterion(out.view(-1, out.size(2)), labels.view(-1))
      #print(out.shape)
      #print(predicted.shape)
      #print(loss.shape)

      loss.backward()
      optimizer.step()

      if batch_idx % 50 == 0:
          print('Epoch', epoch + 1, 'Loss', loss.item(), 'batch_idx', batch_idx)

def val_epoch(model, loader, criterion, optimizer):
  model.eval()
  total_loss = 0.0
  
  total = 0
  total_prec = 0.0
  total_recall = 0.0
  total_acc = 0.0
  tp = 0
  fp = 0
  fn = 0
  tn = 0
  flag = 0
  for batch_idx, (data, labels) in enumerate(loader):
      
      data = data.to(DEVICE)
      labels = labels.to(DEVICE)

      optimizer.zero_grad()
      attn_masks = (data != 0).long()
      attn_masks = attn_masks.to(DEVICE)
      out = model(data, attn_masks)

      _, predicted = torch.max(out.data, 2)
      #if flag == 0:
      #torch.set_printoptions(profile="full")
      #print(predicted.shape)
      #print(predicted)
      #print("helloval: ", torch.nonzero(predicted).shape[0] == 0)

      for i in range(predicted.size(0)):
        for j in range(predicted.size(1)):
          if labels[i][j] != 0 and predicted[i][j] != 0:
            tp += 1
          elif labels[i][j] != 0 and predicted[i][j] == 0:
            fn += 1
          elif labels[i][j] == 0 and predicted[i][j] != 0:
            fp += 1
          else:
            tn += 1

      total += data.size(0)
      loss = criterion(out.view(-1, out.size(2)), labels.view(-1))
      #scheduler.step(loss)
      total_loss += loss.item()

      #if batch_idx % 5 == 0:
        #print('Val Epoch', epoch + 1, 'batch_idx', batch_idx)

  if tp+fp > 0:
    total_prec += tp/(tp + fp)
  if tp+fn > 0:
    total_recall += tp/(tp+fn)

  print("tp: ", tp)
  print("tn: ", tn)
  print("fp: ", fp)
  print("fn: ", fn)
  total_acc = (tp+tn)/(tp+fp+tn+fn)
  total = len(loader)
  print('Val Loss: ' + str(total_loss/total) + ", Precision: "  + str(total_prec) + ", Recall: " + str(total_recall) + ", Accuracy: " + str(total_acc) + "\n")

In [37]:
model = LanguageModel()
model = model.to(DEVICE)

In [38]:
train_data = final_dataset[:6000]
val_data = final_dataset[6000:]

train_dataset = ModelDataset(train_data)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=50)

val_dataset = ModelDataset(val_data)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=50)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=0)

weights = [1.0, 50.0, 50.0]
class_weights = torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
#criterion = nn.CrossEntropyLoss(reduction="none")
criterion = criterion.to(DEVICE)

In [34]:
import numpy as np
a = np.array([len(i[2]) for i in dataset])
b = np.array([i[2] for i in dataset])
c = np.array([i[0] for i in dataset])
d = np.array([i[1].find(i[0]) for i in dataset])

#np.array([i[1] for i in dataset]).shape
#dataset[0][1]
z = None
for i in dataset:
  if len(i[2]) == 571:
    print(i)
    z = i
    break
print(max(a))
print(b)
print(a)
len(a)
d

#for i in a:
#  if i > 70:
#    #print(i)

['infrastructure', 'chapter 8: using google web services169api nameurlcategorygoogle descriptiongoogle apps marketplacehttp://code.google.com/googleapps/marketplace/productivitysell integrated applications to millions of google apps users.gmail apis and toolshttp://code.google.com/apis/gmail/labscreate gadgets for gmail, and interact with the inbox.google base data api (labs)http://code.google.com/apis/base/labsmanage google base content programmatically.blogger data api (labs)http://code.google.com/apis/blogger/labs, socialenable your apps to view and update blogger content.google books search apis (labs)http://code.google.com/apis/books/labs, searchsearch the complete index of book search, and integrate with its social features.google buzz (labs)http://code.google.com/apis/buzz/labs, socialshare updates, photos, vid-eos, and more, and start con-versations about the things you find interesting.google calendar apis and toolshttp://code.google.com/apis/calendar/socialcreate and manage e

array([ 63,  77, 187, ..., 130, 115,  28])

In [39]:
NO_EPOCHS = 40

for epoch in range(NO_EPOCHS):
  train_epoch(model, train_loader, criterion, optimizer)
  val_epoch(model, train_loader, criterion, optimizer)
  val_epoch(model, val_loader, criterion, optimizer)

Epoch 1 Loss 1.1040235757827759 batch_idx 0
Epoch 1 Loss 0.9353756308555603 batch_idx 50
Epoch 1 Loss 0.9041926860809326 batch_idx 100
tp:  37
tn:  470705
fp:  1432
fn:  7826
Val Loss: 0.9281934425234795, Precision: 0.02518720217835262, Recall: 0.00470558311077197, Accuracy: 0.9807125

tp:  14
tn:  155747
fp:  664
fn:  2615
Val Loss: 0.9393446475267411, Precision: 0.02064896755162242, Recall: 0.005325218714340053, Accuracy: 0.9793825452716298

Epoch 2 Loss 0.9142467379570007 batch_idx 0
Epoch 2 Loss 0.8967660069465637 batch_idx 50
Epoch 2 Loss 1.0021799802780151 batch_idx 100
tp:  1
tn:  472072
fp:  65
fn:  7862
Val Loss: 0.9255705873171488, Precision: 0.015151515151515152, Recall: 0.00012717792191275595, Accuracy: 0.9834854166666667

tp:  0
tn:  156388
fp:  23
fn:  2629
Val Loss: 0.9387927427887917, Precision: 0.0, Recall: 0.0, Accuracy: 0.9833249496981892

Epoch 3 Loss 0.9683197140693665 batch_idx 0
Epoch 3 Loss 0.9581891298294067 batch_idx 50
Epoch 3 Loss 0.8922590017318726 batch_id

KeyboardInterrupt: ignored